<a href="https://colab.research.google.com/github/vsathyap/PyTorch_Fundamentals/blob/main/simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial 5: Create a basic Neural Network Model**

In [ ]:
import torch
#To use the neural network from torch
import torch.nn as nn
# To move our data forward
import torch.nn.functional as F

In [ ]:
# Create a Model Class that inherits nn.Module (using OOPs)
class Model(nn.Module):
  '''Input Layer (it has the 4 features of the IRIS dataset)
  --> Hidden Layer 1
  --> Hidden Layer 2
  --> Output Layer (3 classes of the IRIS dataset)'''
  # Constructor of the class
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    # Instantiate our nn.Module
    super().__init__()

    #Fully connected network
    self.fc1 = nn.Linear(in_features,h1) # Input Layer --> H1
    self.fc2 = nn.Linear(h1,h2) # H1 --> H2
    self.fc3 = nn.Linear(h2,out_features) # H2 --> Output Layer

  #Function to move everything forward
  def forward(self,x):
    #Using reLu (rectified Linear Unit), f(x) = max(0,x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))

    return x


In [ ]:
# To keep the random number generator deterministic
torch.manual_seed(41)

In [ ]:
# Create an instance of the model
model = Model()

# **Tutorial 7 : Load the data and train the model**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Create an url to load our data from
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)
my_df

In [ ]:
my_df.head()

In [ ]:
# List the unique labels in species
my_df['species'].unique()

In [ ]:
# To replace the species with integers
my_df['species'] = my_df['species'].replace('setosa',0.0)
my_df['species'] = my_df['species'].replace('virginica',2.0)
my_df['species'] = my_df['species'].replace('versicolor',1.0)

In [ ]:
my_df

In [ ]:
# Train and Test Split
X = my_df.drop('species',axis =1)
Y = my_df['species']

In [ ]:
# Convert them to numpy arrays
X = X.values
Y = Y.values
print(f'X = {X}')
print(f'Y = {Y}')

In [ ]:
# To split the dataset into train and test dataset, we need scikit learn
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 41)

In [ ]:
# Convert features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [ ]:
# Long Tensor = 64 bit integers
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
# Error function to measure the prediction error
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# lower the Learning rate, longer it will take to train.

In [ ]:
# Train our model
# 1 Epoch is 1 run through all the data
epochs = 100
losses = []
for i in range(epochs):
  # Process forward and predict
  y_pred = model.forward(X_train)

  # Calculate the error in prediction
  loss = criterion(y_pred,y_train)

  #Keep track of the loss per epoch
  losses.append(loss.detach().numpy())

  # Print every 10th epoch
  if i % 10==0:
    print(f"Epoch : {i}, Loss : {loss}")

  # Back propagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
# Plot the loss across epochs
plt.plot(range(epochs),losses)
plt.ylabel('Losses')
plt.xlabel('Epochs')


# **Tutorial 7 : Evaluate the test set**

In [ ]:
# Evaluate the Model on the test dataset (do one pass through, wihtout back-propagation)
with torch.no_grad(): # Turn off back propagation
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)


In [ ]:
loss

In [ ]:
# Let's look at how the NN did on test data
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    print(f'{i+1}.) {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct+=1

print(f'We got {correct} correct!')

# **Tutorial 8 : Evaluate the model on a NEW (Validation set)**

In [ ]:
# Create a new data point
new_iris = torch.tensor([4.7,3.2,1.3,0.2])

In [ ]:
# Run it through the model
with torch.no_grad():
  y_eval = model(new_iris)

print(f'Prediction is {y_eval.argmax().item()}')

# **Tutorial 9: Save and Load the NN model**

In [ ]:
# Save our NN Model
torch.save(model.state_dict(),'my_simple_iris_NN.pt')

In [ ]:
# How to load the saved model
new_model = Model()
new_model.load_state_dict(torch.load('my_simple_iris_NN.pt'))

In [ ]:
# to check if the correct model has loaded
new_model.eval()